In [1]:
import getpass
import os

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("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")

In [2]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="mixtral-8x7b-32768",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [3]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
print("\n".join([f"{msg[0]}: {msg[1]}" for msg in messages]))
ai_msg

system: You are a helpful assistant that translates English to French. Translate the user sentence.
human: I love programming.


AIMessage(content='I love programming in French is: "J\'aime programmer."', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 30, 'total_tokens': 46, 'completion_time': 0.02398395, 'prompt_time': 0.002646083, 'queue_time': 0.011554217, 'total_time': 0.026630033}, 'model_name': 'mixtral-8x7b-32768', 'system_fingerprint': 'fp_c5f20b5bb1', 'finish_reason': 'stop', 'logprobs': None}, id='run-7096da94-18fd-4d07-a738-74671951dfc6-0', usage_metadata={'input_tokens': 30, 'output_tokens': 16, 'total_tokens': 46})

In [4]:
messages = [
    {"role": "system", "content": "You are a helpful assistant that converts unstructured prompts to structured json prompts"},
    {"role": "user", "content": "I want to predict the sentiment of a sentence."}
]

ai_msg = llm.invoke(messages)
ai_msg


AIMessage(content='Sure, I can help you structure a JSON prompt for sentiment analysis. Here\'s an example of how you might structure the JSON prompt:\n\n```json\n{\n  "text": "Your sentence goes here"\n}\n```\n\nYou can replace "Your sentence goes here" with the sentence you want to analyze for sentiment. The model will then return a prediction of the sentiment of the sentence, which could be positive, negative, or neutral.\n\nHere\'s an example of what the JSON prompt might look like with a specific sentence:\n\n```json\n{\n  "text": "I had a great day at the park!"\n}\n```\n\nIn this example, the model might predict a positive sentiment for the sentence, since it expresses a positive experience.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 169, 'prompt_tokens': 36, 'total_tokens': 205, 'completion_time': 0.268401265, 'prompt_time': 0.003204761, 'queue_time': 0.011328069, 'total_time': 0.271606026}, 'model_name': 'mixtral-8x7b-32768', 'system_finger

In [5]:
messages = [
    {
        "role": "system",
        "content": "You are a medical assistant that extracts structured information from unstructured clinical notes. You will identify patient complaints, investigations, and medication information."
    },
    {
        "role": "user",
        "content": "Patient Complaints of High grade fever, chest pain, radiating towards right shoulder. Sweating, Palpitation since 5 days. Advice investigation: CBC, LFT, Chest X-ray, Abdomen Ultrasound. Medication: Diclofenac 325mg twice a day for 5 days, Amoxiclave 625mg once a day for 5 days, Azithromycin 500mg Once a day, Ibuprofen SOS, Paracetamol sos, Pentoprazol before breakfast."
    }
]


In [12]:
import json
import ast

messages = [
    {
        "role": "system",
        "content": """
        You are a medical assistant capable of extracting structured details from unstructured clinical text. Your task is to identify specific categories of information from the provided text. Additionally, you need to identify and flag any unknown or unfamiliar terms that may need further clarification or special handling.

        The categories you need to extract from the text are:
        1. **Status**: Identify the patient's health status, such as "stable," "critical," "recovering," "improving," etc.
        2. **Pharmacy**: Extract references to medication, prescriptions, or pharmacy-related details (e.g., medication names, dosages, directions).
        3. **Services**: Extract any references to medical services, tests, or procedures mentioned (e.g., lab tests, imaging, surgeries).
        4. **Unknown Words**: Flag any terms or phrases that are unfamiliar or do not fit within the recognized categories, indicating the need for further clarification or special handling.

        Please note that further clarification or special handling may be needed for the patient's health status and the interpretation of their symptoms.
    
        """
    },
    {
        "role": "user",
        "content": """Patient complains of abdominal pain predominantly in the right lower quadrant, fever, and loss of appetite over two days. Reports nausea and occasional vomiting.
Advice investigation: CBC, CRP, Abdominal Ultrasound, Urinalysis.
Medication: Metronidazole 400mg twice a day for 7 days, Cefixime 200mg twice a day for 5 days, Pantoprazole 40mg once daily before breakfast.
        """
    },
    {
        "role":"system",
        "content":""" Based ono the provided information, you can provide a structured response. Here is the response:
        {
    "status": "stable",
    "pharmacy": {
        "medications": [
            {
                "name": medicine name,  // This is a literal string, not a placeholder
                "dosage": amount,
                "unit" : amount    // Same issue with placeholders
                "frequency": frequency
            },
        ]
    }, ```Observe carefully the format of the response, extracting all the services```
    "services": {  
        "tests": [
           // Provide all the tests that are being prescribed to the patient, if any
        ]           
    },
    "unknown_words": [ 
    ]
}
        """

    },

    {
        "role":"user",
        "content": """  Take the response, provide before and provide it in json format, with an indent of 4 spaces  taking the responses you have provided and provide it in json format. 
        """
    }
]


ai_msg = llm.invoke(messages)

print(ai_msg)


# content=ai_msg.content
# json_str = re.search(r'```json\n(.*?)```', content, re.DOTALL).group(1)
# ai_msg_json = json.loads(json_str)
# print(ai_msg_json)
# formatted_json = json.dumps(ai_msg_json, indent=8)
# print(formatted_json)

def extract_json(content):
    start = content.find('```json\n') + 7
    end = content.find('```', start)
    json_str = content[start:end].strip()
    return ast.literal_eval(json_str)
def extract_json_manual(content):
    start = content.find('```json\n') + 7
    end = content.find('```', start)
    json_str = content[start:end].strip()
    import json 
    return json.loads(json_str)
ai_msg_json=extract_json(ai_msg.content)
ai_msg_json= extract_json_manual(ai_msg.content)


print(json.dumps(ai_msg_json, indent=4))

content='Here is the structured response in JSON format with an indent of 4 spaces:\n\n```json\n{\n    "status": "stable",\n    "pharmacy": {\n        "medications": [\n            {\n                "name": "Metronidazole",\n                "dosage": 400,\n                "unit": "mg",\n                "frequency": "twice a day"\n            },\n            {\n                "name": "Cefixime",\n                "dosage": 200,\n                "unit": "mg",\n                "frequency": "twice a day"\n            },\n            {\n                "name": "Pantoprazole",\n                "dosage": 40,\n                "unit": "mg",\n                "frequency": "once daily before breakfast"\n            }\n        ]\n    },\n    "services": {\n        "tests": [\n            "CBC",\n            "CRP",\n            "Abdominal Ultrasound",\n            "Urinalysis"\n        ]\n    },\n    "unknown_words": []\n}\n```' additional_kwargs={} response_metadata={'token_usage': {'completion_to